应用系统在日常运行时，会对底层软、硬件造成负荷，影响应用系统性能。本次重点分析存储设备中磁盘容量预测，通过对磁盘容量进行预测，可预测磁盘未来的负载情况，避免应用系统因出现存储容量耗尽的情况导致系统负载率过高，最终引发系统故障。
SYS_NAME:资产所在的系统名称；
NAME:资产名称；
TARGET_ID：属性标识号，183、184分别标识磁盘容量大小和磁盘已使用大小；
DESCRIPTION:针对属性标识的说明；
ENTITY:具体的属性；
VALUE:采集到的值；
COLLECTTIME:采集的时间

In [13]:
import pandas as pd
data=pd.read_excel('./discdata.xls')
data=data[data['TARGET_ID']==184].copy()
data_group=data.groupby('COLLECTTIME')
#定义属性变换函数
def attr_trans(x):
    result=pd.Series(index=['SYS_NAME','CWXT_DB:184:C:\\','CWXT_DB:184:D:\\','COLLECTTIME'])
    result['SYS_NAME']=x['SYS_NAME'].iloc[0]
    result['COLLECTTIME']=x['COLLECTTIME'].iloc[0]
    result['CWXT_DB:184:C:\\']=x['VALUE'].iloc[0]
    result['CWXT_DB:184:D:\\']=x['VALUE'].iloc[1]
    return result
data_processed=data_group.apply(attr_trans)
data_processed.to_excel('./discdata_processed.xlsx',index=False)

In [23]:
#平稳性检测
data=pd.read_excel('./discdata_processed.xlsx')
data=data.iloc[:len(data)-5]
from statsmodels.tsa.stattools import adfuller as ADF
diff=0
adf=ADF(data['CWXT_DB:184:D:\\'])
while adf[1]>=0.05:
    diff+=1
    adf=ADF(data['CWXT_DB:184:D:\\'].diff(diff).dropna())
print('原始序列经过%s阶差分后归于平稳，p值为%s' %(diff,adf[1]))

原始序列经过1阶差分后归于平稳，p值为4.792591263393756e-07


In [25]:
#白噪声检测
from statsmodels.stats.diagnostic import acorr_ljungbox
[[lb],[p]]=acorr_ljungbox(data['CWXT_DB:184:D:\\'],lags=1)
if p<0.05:
    print('原始序列为非白噪声序列，对应的p值为：%s' %p)
else:
    print('原始序列为白噪声序列，对应的p值为：%s' %p)
[[lb],[p]]=acorr_ljungbox(data['CWXT_DB:184:D:\\'].diff().dropna(),lags=1)
if p<0.05:
    print('一阶差分序列为非白噪声序列，对应的p值为：%s' %p)
else:
    print('一阶差分序列为白噪声序列，对应的p值为：%s' %p)

原始序列为非白噪声序列，对应的p值为：9.95850372977218e-06
一阶差分序列为白噪声序列，对应的p值为：0.1143302597764247


In [51]:
#确定最佳p,d,q值
data=pd.read_excel('./discdata_processed.xlsx',index_col='COLLECTTIME')
data=data.iloc[:len(data)-5]
xdata=data['CWXT_DB:184:D:\\']
from statsmodels.tsa.arima_model import ARIMA
pmax=int(len(xdata)/10)
qmax=int(len(xdata)/10)
bic_matrix=[]
for p in range(qmax+1):
    tmp=[]
    for q in range(pmax+1):
        try:
            tmp.append(ARIMA(xdata,(p,1,q)).fit().bic)
        except:
            tmp.append(None)
    bic_matrix.append(tmp)
bic_matrix=pd.DataFrame(bic_matrix)
bic_matrix=bic_matrix.astype('float64')
p,q=bic_matrix.stack().idxmin()
print('BIC最小的p值和q值为：%s、%s' %(p,q))

BIC最小的p值和q值为：1、1


In [52]:
#模型检验
from statsmodels.tsa.arima_model import ARIMA
arima=ARIMA(xdata,(0,1,1)).fit()
xdata_pred=arima.predict(typ='levels')
pred_error=(xdata_pred-xdata).dropna()
from statsmodels.stats.diagnostic import acorr_ljungbox
lb,p=acorr_ljungbox(pred_error,lags=12)
h=(p<0.05).sum()
if h>0:
    print('模型ARIMA(0,1,1)不符合白噪声检验')
else:
    print('模型ARIMA(0,1,1)符合白噪声检验')

D:\winapps\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
D:\winapps\anaconda\lib\site-packages\statsmodels\tsa\base\tsa_model.py:191: FutureWarning: Creating a DatetimeIndex by passing range endpoints is deprecated.  Use `pandas.date_range` instead.
  start=index[0], end=index[-1], freq=freq)


模型ARIMA(0,1,1)符合白噪声检验


In [57]:
#计算预测误差
data=pd.read_excel('./predictdata.xlsx')
abs_=(data[u'预测值']-data[u'实际值']).abs()
mae_=abs_.mean()
rmse_=((abs_**2).mean())
mape_=(abs_/data[u'实际值']).mean()
print('平均绝对误差为：%.4f,均方误差为：%.4f,平均绝对百分误差为：%.6f' %(mae_,rmse_,mape_))

平均绝对误差为：1.1068,均方误差为：1.3744,平均绝对百分误差为：0.012610


结合实际业务分析，将误差阈值设定为1.5，实际值与预测值之间的误差都小于误差阈值，模型的预测效果在实际业务可接受范围内，可以采用此模型进行预测。